# Imports

In [2]:
import anki_vector
from anki_vector.util import degrees, distance_mm, speed_mmps, Pose
from OpenGL.GL import *
import time
import math

# Connections

In [3]:
# Create a Robot object
robot = anki_vector.Robot(show_viewer=True,
                          show_3d_viewer=True,
                       enable_nav_map_feed=True)

'''
robot = anki_vector.Robot(show_viewer=True,
                       show_3d_viewer=True,
                       enable_camera_feed=True,
                       enable_face_detection=True,
                       enable_custom_object_detection=True,
                       enable_nav_map_feed=True)
'''
# Connect to the Robot
robot.connect()

2019-04-27 20:07:30,619 anki_vector.robot.Robot WARNING  No serial number provided. Automatically selecting 00400933
2019-04-27 20:07:30,624 anki_vector.robot.Robot WARNING  enable_camera_feed should be True for viewer to render correctly.
2019-04-27 20:07:30,633 anki_vector.connection.Connection INFO     Connecting to 10.0.0.43:443 for Vector-D8H6 using C:\Users\darrylsw\.anki_vector\Vector-D8H6-00400933.cert
2019-04-27 20:07:36,891 anki_vector.connection.Connection INFO     control_granted_response {
}



In [4]:
robot.disconnect()

# Constants

In [ ]:
PROXIMITY_SCAN_DISTANCE_THRESHOLD_MM = 300
PROXIMITY_SCAN_OPEN_NODE_DISTANCE_MM = 230
PROXIMITY_SCAN_SAMPLE_FREQUENCY_HZ = 15.0
ROXIMITY_SCAN_TURN_DURATION_S = 10.0
PROXIMITY_SCAN_BIND_THRESHOLD_MM = 30.0
PROXIMITY_EXPLORATION_SHUTDOWN_DELAY_S = 8.0

# Functions

In [135]:
class SearchNode():
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y

def scan(robot: anki_vector.robot.Robot):
    print('scanning')
    time_scan = 10.0
    robot.behavior.turn_in_place(angle=degrees(360.0), speed=degrees(360.0 / time_scan))
    
def get_surrounding_nodes(robot: anki_vector.robot.Robot):
    print('get surrounding nodes')
    search_point = 10
    distance = 150
    angle_incre = 6.283 / search_point
    
    pos = robot.pose.position
    ang = robot.pose.rotation.angle_z.radians
    
    surrounding = []
    
    print('current:', pos)
    for i in range(0, search_point):
        pot_x = pos.x + distance * math.sin(ang)
        pot_y = pos.y + distance * math.cos(ang)
        explore_loc = SearchNode(pot_x, pot_y)
        surrounding.append(explore_loc)
        ang += angle_incre
        
    surrounding.reverse()
        
    return surrounding # So the frontiest node gets to the top of the stack
    
def get_explorable_nodes(robot: anki_vector.robot.Robot, explorable, explored):
    candidate_nodes = get_surrounding_nodes(robot)
    curr_map = robot.nav_map.latest_nav_map
    
    # clear explorables from explored
    explorable[:] = [candidate for candidate in explorable if not node_explored(candidate, explored, robot)]
    
    for node in candidate_nodes:
        if curr_map.get_content(node.x, node.y) == 1 or curr_map.get_content(node.x, node.y) == 2:
            if node_fits_robot(node, robot) and not node_explored(node, explored, robot):
                explorable.append(node)
                
    return explorable
            

def node_fits_robot(node: SearchNode, robot: anki_vector.robot.Robot):
    curr_map = robot.nav_map.latest_nav_map
    robot_size = 50
    
    # Check the four directions of the robot
    eval_x = node.x + robot_size
    eval_y = node.y + robot_size
    if curr_map.get_content(eval_x, eval_y) != 1 and curr_map.get_content(eval_x, eval_y) != 2:
        return False
    
    eval_x = node.x - robot_size
    eval_y = node.y + robot_size
    if curr_map.get_content(eval_x, eval_y) != 1 and curr_map.get_content(eval_x, eval_y) != 2:
        return False
    
    eval_x = node.x + robot_size
    eval_y = node.y - robot_size
    if curr_map.get_content(eval_x, eval_y) != 1 and curr_map.get_content(eval_x, eval_y) != 2:
        return False
    
    eval_x = node.x - robot_size
    eval_y = node.y - robot_size
    if curr_map.get_content(eval_x, eval_y) != 1 and curr_map.get_content(eval_x, eval_y) != 2:
        return False
    
    return True

def node_explored(node: SearchNode, explored_nodes, robot: anki_vector.robot.Robot):
    radius = 150
    curr_map = robot.nav_map.latest_nav_map
    
    for explored in explored_nodes:
        distance = math.sqrt((explored.x - node.x) ** 2 + (explored.y - node.y) ** 2)
        if distance < radius:
            if curr_map.get_content(node.x, node.y) == 1 or curr_map.get_content(node.x, node.y) == 2:
                return True
    
    return False

def get_angle_from_to(source: SearchNode, dest: SearchNode):
    x = dest.x - source.x
    y = dest.y - source.y
    return math.atan2(y , x)

def drive_to(dest: SearchNode, robot: anki_vector.robot.Robot):
    posi = robot.pose.position
    angle = get_angle_from_to(SearchNode(posi.x, posi.y), dest)
    
    pose = Pose(x=dest.x, y=dest.y, z=0, angle_z=anki_vector.util.Angle(radians=angle))
    robot.behavior.go_to_pose(pose)
    
    
def clear_explorables(explorable, explored):
    print('placeholder')
    

In [111]:
# Begin
robot.nav_map.init_nav_map_feed(frequency=0.5)
print('driving off')
robot.behavior.drive_off_charger()
print('setting lift')
robot.behavior.set_lift_height(0.0)

driving off
setting lift


status {
  code: RESPONSE_RECEIVED
}
result {
}

In [ ]:
#robot.pose.rotation.angle_z.degrees
robot.pose.rotation.angle_z.radians

In [133]:
# End
scan(robot)

scanning


In [ ]:
robot.pose.position

In [ ]:
position = robot.pose.position
pose = Pose(x=position.x, y=position.y, z=position.z, angle_z=anki_vector.util.Angle(radians=700.10665441777373975))
robot.behavior.go_to_pose(pose)

In [113]:
posi = robot.pose.position
explored = []
explorable = []
explored.append(SearchNode(posi.x, posi.y))
explorable = get_explorable_nodes(robot, explorable, explored)
print_exp(explorable)

get surrounding nodes
current: <Position x: 124.42 y: 6.23 z: 0.00>
( 260.10989071045907 , -57.71027137668853 )
( 271.7805207133544 , 34.254621441118196 )


In [121]:
posi = robot.pose.position
explored.append(SearchNode(posi.x, posi.y))

explorable = get_explorable_nodes(robot, explorable, explored)

print_exp(explorable)

get surrounding nodes
current: <Position x: 421.52 y: 59.16 z: 0.00>
( 352.06702757829055 , 192.1076631829535 )
( 399.5516977660236 , -89.22667558345964 )
( 490.96197445875083 , -73.80245986225634 )
( 555.8495421064243 , -7.595597072785864 )
( 569.4309688184246 , 84.10658373073557 )
( 526.5189070627569 , 166.2789639538551 )
( 443.5033711112195 , 207.53627732171975 )


In [71]:
def print_exp(explorable):
    for e in explorable:
        print('(', e.x,',', e.y, ')')

In [131]:
drive_to(explorable[5], robot)


In [132]:
drive_to(explorable[2], robot)